In [22]:
"""
Define optimisation constraints from PyPSA networks with Linopy.
"""
import gc
import logging
import pypsa
import pandas as pd
from linopy import LinearExpression, Variable, merge
from numpy import arange, cumsum, inf, isfinite, nan, roll
from scipy import sparse
from xarray import DataArray, Dataset, concat

from pypsa.descriptors import (
    additional_linkports,
    expand_series,
    get_activity_mask,
    get_bounds_pu,
)
from pypsa.descriptors import get_switchable_as_dense as get_as_dense
from pypsa.descriptors import nominal_attrs
from pypsa.optimization.common import reindex
from pypsa.optimization.compat import define_constraints, get_var, linexpr

from _helpers import remove_leap_day

from solve_network_dispatch import set_existing_committable, set_max_status, set_upper_combined_status_bus, set_upper_avg_status_over_sns
from prepare_and_solve_network import ccgt_steam_constraints

logger = logging.getLogger(__name__)

In [23]:
n = pypsa.Network("../networks/solved_val-LC-UNC_30-supply_redz_lcopt_LC-2000SEG.nc")

INFO:pypsa.io:Imported network solved_val-LC-UNC_30-supply_redz_lcopt_LC-2000SEG.nc has buses, carriers, generators, global_constraints, lines, loads, storage_units


In [24]:
n_seg = pypsa.Network("../networks/solved_val-LC-UNC_30-supply_redz_lcopt_LC-2000SEG.nc")

INFO:pypsa.io:Imported network solved_val-LC-UNC_30-supply_redz_lcopt_LC-2000SEG.nc has buses, carriers, generators, global_constraints, lines, loads, storage_units


In [30]:
print(n.generators.p_nom_opt.groupby(n.generators.carrier).sum())
print(n.storage_units.p_nom_opt.groupby(n.storage_units.carrier).sum())

carrier
battery          3.212803e+04
biomass          3.536078e-02
ccgt_steam       3.225690e+02
coal             3.850602e+04
hydro            6.830200e+02
hydro_import     1.764000e+03
load_shedding    3.000000e+07
nuclear          1.854005e+03
ocgt_diesel      3.077915e+03
ocgt_gas         3.420000e+02
solar_csp        5.000000e+02
solar_pv         5.080778e+03
wind             6.962218e+03
Name: p_nom_opt, dtype: float64
carrier
battery     733.998045
phs        2912.078305
Name: p_nom_opt, dtype: float64


In [31]:
print(n_seg.generators.p_nom_opt.groupby(n_seg.generators.carrier).sum())
print(n_seg.storage_units.p_nom_opt.groupby(n_seg.storage_units.carrier).sum())

carrier
battery          3.212803e+04
biomass          3.536078e-02
ccgt_steam       3.225690e+02
coal             3.850602e+04
hydro            6.830200e+02
hydro_import     1.764000e+03
load_shedding    3.000000e+07
nuclear          1.854005e+03
ocgt_diesel      3.077915e+03
ocgt_gas         3.420000e+02
solar_csp        5.000000e+02
solar_pv         5.080778e+03
wind             6.962218e+03
Name: p_nom_opt, dtype: float64
carrier
battery     733.998045
phs        2912.078305
Name: p_nom_opt, dtype: float64


In [27]:
n.loads_t.p_set.loc[2040].sum().sum()/1e6

73.54116396611798

In [28]:
n.

SyntaxError: invalid syntax (395983660.py, line 1)

In [ ]:
n.statistics()["Supply"]/1e6

2025        2030        2035  \
Generator   Battery               2.177650e+02  239.369770  256.483804   
            Biomass               2.727697e-05    0.000068    0.000138   
            Ccgt_Steam            1.495407e-05    0.000046    0.000812   
            Coal                  1.500321e+00    1.310813    1.539587   
            Hydro                 3.946491e-02    0.047307    0.094934   
            Hydro_Import          4.163327e-01    0.394801    0.475383   
            Load_Shedding         8.551162e-07    0.000001    0.000023   
            Nuclear               6.888755e+00    5.419081    7.297693   
            Ocgt_Diesel           4.550166e-04    0.000714    0.032349   
            Ocgt_Gas              2.806421e-06    0.000000    0.000000   
            Solar_Csp             2.091701e-05    0.000381    0.006670   
            Solar_Pv              9.832474e+00    9.808000   10.189313   
            Wind                  2.249044e+01   22.495407   19.770686   
Line        AC                    6.215996e+02  690.943414  743.085913   
Load        -                     0.000000e+00    0.000000    0.000000   
StorageUnit Battery               1.127113e-03    0.002979    0.077657   
            Pumped Hydro Storage  5.518963e+00    5.829879    5.498736   

                                        2040  
Generator   Battery               270.654966  
            Biomass                 0.000286  
            Ccgt_Steam              0.032191  
            Coal                    3.275032  
            Hydro                   0.169742  
            Hydro_Import            1.074413  
            Load_Shedding           0.001394  
            Nuclear                12.978293  
            Ocgt_Diesel             0.107450  
            Ocgt_Gas                0.000000  
            Solar_Csp               0.015639  
            Solar_Pv                9.013453  
            Wind                   15.660521  
Line        AC                    782.796453  
Load        -                       0.000000  
StorageUnit Battery                 1.916544  
            Pumped Hydro Storage    4.546031